# StyleGAN2-ADA-PyTorch

**Notes**
* Training and Inference sections should be fairly stable. I’ll slowly add new features but it should work for most mainstream use cases.
* Advanced Features are being documented toward the bottom of this notebook

---

If you find this notebook useful, consider signing up for my [Patreon](https://www.patreon.com/bustbright) or [YouTube channel](https://www.youtube.com/channel/UCaZuPdmZ380SFUMKHVsv_AA/join). You can also send me a one-time payment on [Venmo](https://venmo.com/Derrick-Schultz).

## Setup

Let’s start by checking to see what GPU we’ve been assigned. Ideally we get a V100, but a P100 is fine too. Other GPUs may lead to issues.

In [ ]:
!nvidia-smi -L

Next let’s connect our Google Drive account. This is optional but highly recommended.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!sudo apt install python3.7

In [ ]:
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3
!sudo apt install python3-pip
!sudo apt install python3.7-distutils

In [ ]:
!python --version
!pip --version

## Install repo

The next cell will install the StyleGAN repository in Google Drive. If you have already installed it it will just move into that folder. If you don’t have Google Drive connected it will just install the necessary code in Colab.

In [ ]:
import os
!pip install gdown --upgrade

if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

In [ ]:
#Uninstall new JAX
!pip uninstall jax jaxlib -y
#GPU frontend
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
#CPU frontend
#!pip install jax[cpu]==0.3.10
#Downgrade Pytorch
!pip uninstall torch torchvision -y
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install timm==0.4.12 ftfy==6.1.1 ninja==1.10.2 opensimplex

You probably don’t need to run this, but this will update your repo to the latest and greatest.

In [ ]:
%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

## Dataset Preparation

Upload a .zip of square images to the `datasets` folder. Previously you had to convert your model to .tfrecords. That’s no longer needed :)

## Train model

In [ ]:
!python stylegan2-ada-pytorch/train.py \
 --outdir={output_folder} \
 --snap={snap} \
 --metrics=none \
 --data={'/content/drive/MyDrive/face_img.zip'} \
 --cfg=auto_norp \
 --cifar_tune=1 \
 --gamma={gamma} \
 --kimg={kimg} \
 --batch={mini_batch} \
 --cfg_map=8 \
 --augpipe=bg \
 --freezed=10 \
 --resume=ffhq_1024 \


Below are a series of variables you need to set to run the training. You probably won’t need to touch most of them.

* `dataset_path`: this is the path to your .zip file
* `resume_from`: if you’re starting a new dataset I recommend `'ffhq1024'` or `'./pretrained/wikiart.pkl'`
* `mirror_x` and `mirror_y`: Allow the dataset to use horizontal or vertical mirroring.

In [ ]:
#required: definitely edit these!
dataset_path = '/content/drive/MyDrive/face_img.zip'
resume_from = '/content/drive/MyDrive/ffhq.pkl'
aug_strength = 0.328
train_count = 10
mirror_x = True
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = 'auto'
snapshot_count = 1

In [ ]:
!pip install click requests tqdm pyspng psutil tensorboard ninja imageio-ffmpeg==0.4.3

In [ ]:
!sudo apt install python3.7-dev

In [ ]:
!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./results2 --data=$dataset_path --snap=$snapshot_count --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count --freezed=4

In [ ]:
#required: definitely edit these!
dataset_path = '/content/drive/MyDrive/resized.zip'
resume_from = '/content/drive/MyDrive/ffhq_1024.pkl'
aug_strength = 0.328
train_count = 100
mirror_x = False
#mirror_y = False

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = 'paper256'
snapshot_count = 4

### Resume Training

Once Colab has shutdown, you’ll need to resume your training. Reset the variables above, particularly the `resume_from` and `aug_strength` settings.

1. Point `resume_from` to the last .pkl you trained (you’ll find these in the `results` folder)
2. Update `aug_strength` to match the augment value of the last pkl file. Often you’ll see this in the console, but you may need to look at the `log.txt`. Updating this makes sure training stays as stable as possible.
3. You may want to update `train_count` to keep track of your training progress.

Once all of this has been reset, run that variable cell and the training command cell after it.